In [1]:
import os
import PyPDF2
import pandas as pd

import numpy as np
import nltk 
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize

data = pd.read_csv("Data_mesh_cleaned.csv")
pdf_data = data.copy()


C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\sitas\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
pdf_data

,pdf_name,pdf_content
0,0670.pdf,introducing paradigm smart city platform desig...
1,1-s2.0-S1877050921022365-main.pdf,available procedia computer science author pub...
2,1-s2.0-S1877050923006099-main.pdf,available procedia computer science author pub...
3,2302.01713.pdf,best avoid mess jan bode ibm germany niklas kü...
4,2304.01062.pdf,systematic gray literature review abel goedege...
5,978-1-4842-9253-2.pdf,fabric approach ai guide ai based cataloging g...
6,978-3-031-12423-5_7.pdf,cok survey privacy challenge relation mesh nik...
7,978-3-031-36118-0.pdf,lecture note engineering communication technol...
8,978-3-031-39847-6_1.pdf,integration revitalized warehouse lake robert ...
9,978-3-031-45021-1_23.pdf,converging microservice principle uni ed logic...


In [3]:
from bertopic import BERTopic



topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(pdf_data['pdf_content'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-10-29 21:20:09,121 - BERTopic - Transformed documents to Embeddings
2023-10-29 21:20:18,750 - BERTopic - Reduced dimensionality
2023-10-29 21:20:18,768 - BERTopic - Clustered reduced embeddings


In [4]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,27,-1_product_system_model_architecture


In [5]:
pdf_contents = []
topic_groups = []
top_topic_words = []
dominant_words = []

# Loop through the documents and their corresponding topics
for i, (topic, prob) in enumerate(zip(topics, probs)):
    pdf_contents.append(pdf_data['pdf_content'].iloc[i])  # Extract the original 'pdf_content'
    
    # Extract the topic group (label with highest probability)
    topic_group = topic
    topic_groups.append(topic_group)
    
    # Get the top words for the topic group
    top_words = topic_model.get_topic(topic_group)
    top_topic_words.append(top_words)
    
    # Get the dominant word for the document
    dominant_word = top_words[0][0]  # The most probable word in the top words
    dominant_words.append(dominant_word)

# Create a DataFrame to store the results
result_df = pd.DataFrame({
    'pdf_content': pdf_contents,
    'topic_group': topic_groups,
    'top_topic_words': top_topic_words,
    'dominant_word': dominant_words
})

In [6]:
result_df

,pdf_content,topic_group,top_topic_words,dominant_word
0,introducing paradigm smart city platform desig...,-1,"[(product, 0.03540079799650228), (system, 0.03...",product
1,available procedia computer science author pub...,-1,"[(product, 0.03540079799650228), (system, 0.03...",product
2,available procedia computer science author pub...,-1,"[(product, 0.03540079799650228), (system, 0.03...",product
3,best avoid mess jan bode ibm germany niklas kü...,-1,"[(product, 0.03540079799650228), (system, 0.03...",product
4,systematic gray literature review abel goedege...,-1,"[(product, 0.03540079799650228), (system, 0.03...",product
5,fabric approach ai guide ai based cataloging g...,-1,"[(product, 0.03540079799650228), (system, 0.03...",product
6,cok survey privacy challenge relation mesh nik...,-1,"[(product, 0.03540079799650228), (system, 0.03...",product
7,lecture note engineering communication technol...,-1,"[(product, 0.03540079799650228), (system, 0.03...",product
8,integration revitalized warehouse lake robert ...,-1,"[(product, 0.03540079799650228), (system, 0.03...",product
9,converging microservice principle uni ed logic...,-1,"[(product, 0.03540079799650228), (system, 0.03...",product


,pdf_name,pdf_content,topic,topic_words,dominant_word
0,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...,1,"data, g12, g1, g9, g10",g1
1,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...,1,"data, g12, g1, g9, g10",g1
2,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...,1,"data, g12, g1, g9, g10",g1
3,Adopting_Data_Mesh_principles_to_Boost_Data_Sh...,Adopting Data Mesh principles to Boost Data\nS...,1,"data, g12, g1, g9, g10",g1
4,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...,2,"data, tactical, coalition, military, layer",coalition
5,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...,2,"data, tactical, coalition, military, layer",coalition
6,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...,2,"data, tactical, coalition, military, layer",coalition
7,A_Data_Mesh_Approach_for_Enabling_Data-Centric...,A Data Mesh Approach for Enabling Data-Centric...,2,"data, tactical, coalition, military, layer",coalition
8,datamesh - Copy (2).pdf,Data Mesh: Best Practices to Avoid the Data Me...,0,"data, mesh, interviewees, organizations, domain",interviewees
9,datamesh - Copy (3).pdf,Data Mesh: Best Practices to Avoid the Data Me...,0,"data, mesh, interviewees, organizations, domain",interviewees
